# Text Mining: Sentiment Analysis
Comparing Transformer Sentiment Analysis to traditional Sentiment Analysis. 
Using Text mining

### Dataset: 
SST2 Movie Reviews

In [1]:
# installs Run once
!pip install -q datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00a 0:00:01


In [2]:
!pip install -q spacy scikit-learn
!python -m spacy download en_core_web_md

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 15.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
# Setting up dataset with same format as previous work

import numpy as np
import evaluate
from datasets import load_dataset


dataset = load_dataset("stanfordnlp/sst2")

print("Original dataset structure:")
print(dataset)

# Create test set
train_valid_test = dataset["train"].train_test_split(test_size=0.1, seed=42)
train_valid = train_valid_test["train"].train_test_split(test_size=0.1, seed=42)

dataset["train"] = train_valid["train"]
dataset["validation"] = train_valid["test"]  # Use as validation
dataset["test"] = train_valid_test["test"]   # Use as test

print("\nDataset structure:")
print(f"  Train: {len(dataset['train'])} samples")
print(f"  Validation: {len(dataset['validation'])} samples")
print(f"  Test: {len(dataset['test'])} samples")

2026-01-02 20:19:34.567943: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767385174.781990      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767385174.847875      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767385175.395438      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767385175.395479      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767385175.395481      55 computation_placer.cc:177] computation placer alr

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Original dataset structure:
DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

Dataset structure:
  Train: 54552 samples
  Validation: 6062 samples
  Test: 6735 samples


Printing dataset structure for preprocessing

In [4]:
print(dataset['train'][0])
print(dataset['train'][1])
print(dataset['train'][2])
print(dataset['train'][3])
print(dataset['train'][4])
print(dataset['train'][5])

{'idx': 26396, 'sentence': 'negligible british comedy . ', 'label': 0}
{'idx': 5753, 'sentence': 'has no snap ', 'label': 0}
{'idx': 21514, 'sentence': 'the lines work ', 'label': 1}
{'idx': 21179, 'sentence': 'is a lumbering load of hokum but ', 'label': 0}
{'idx': 44976, 'sentence': 'right film ', 'label': 1}
{'idx': 34302, 'sentence': 'something rare and riveting : ', 'label': 1}


Using SpaCy and sci-kit learn for traditional ML model approach

In [5]:
# Imports
import spacy
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

# set up my data
train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']

# extract labels and sentences
train_sentences = train_data['sentence']
train_labels = train_data['label']

val_sentences = val_data['sentence']
val_labels = val_data['label']

test_sentences = test_data['sentence']
test_labels = test_data['label']

print(f"Training samples: {len(train_sentences)}")
print(f"Validation samples: {len(val_sentences)}")

Training samples: 54552
Validation samples: 6062


In [6]:
# Feature Extraction

nlp = spacy.load("en_core_web_md")

# Chose TF-IDF method to extract notable features for training models
def spacy_tokenizer(sentence):
    doc = nlp(sentence)
    
    tokens = [token.lemma_.lower() for token in doc 
              if not token.is_stop and not token.is_punct and not token.is_space]
    return tokens

# TF-IDF
tfidf_vectorizer = TfidfVectorizer(
    tokenizer=spacy_tokenizer,
    max_features=10000,
    ngram_range=(1, 2)
)

print("Extracting TF-IDF features...")
X_train_tfidf = tfidf_vectorizer.fit_transform(train_sentences)
X_val_tfidf = tfidf_vectorizer.transform(val_sentences)
X_test_tfidf = tfidf_vectorizer.transform(test_sentences)

print(f"TF-IDF shape: {X_train_tfidf.shape}")

Extracting TF-IDF features...


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TF-IDF shape: (54552, 10000)


### Train Models
2 Models selected: 
1. SVM
2. Logistic Regression

In [ ]:
def logistic_regression(X_train, X_val, y_train, y_val, feature_name):
    results = {}
    print(f"Training Logistic Regression with {feature_name}...")
    model = LogisticRegression(max_iter = 1000, random_state = 42)
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    accuracy = accuracy_score(y_val, predictions)
    
    print(f"Logistic Regression Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_val, predictions, target_names=['Negative', 'Positive']))
    
    results['logistic_regression'] = {
        'model': model,
        'accuracy': accuracy,
        'predictions': predictions
    }
    return results

def svm(X_train, X_val, y_train, y_val, feature_name):
    results = {}
    print(f"Training SVM with {feature_name}...")
    
    model = SVC(kernel = 'rbf', C = 1.0, random_state = 42)
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_val)
    accuracy = accuracy_score(y_val, predictions)
    
    print(f"SVM Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_val, predictions, target_names=['Negative', 'Positive']))
    
    results['svm'] = {
        'model': model,
        'accuracy': accuracy,
        'predictions': predictions
    }
    
    return results



# Convert labels to numpy array
y_train = np.array(train_labels)
y_val = np.array(val_labels)

# Train with TF-IDF features
lr_tfidf = logistic_regression(X_train_tfidf, X_val_tfidf, y_train, y_val, "TF-IDF Features")
svm_tfidf = svm(X_train_tfidf, X_val_tfidf, y_train, y_val, "TF-IDF Features")


# Results
print(f"\nTF-IDF Features:")
print(f"  Logistic Regression: {lr_tfidf['logistic_regression']['accuracy']:.4f}")
print(f"  SVM:                 {svm_tfidf['svm']['accuracy']:.4f}")

In [7]:
# Looking at Test data for final eval

def svm(X_train, X_val, y_train, y_val, feature_name):
    results = {}
    print(f"Training SVM with {feature_name}...")
    
    model = SVC(kernel = 'rbf', C = 1.0, random_state = 42)
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_val)
    accuracy = accuracy_score(y_val, predictions)
    
    print(f"SVM Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_val, predictions, target_names=['Negative', 'Positive']))
    
    results['svm'] = {
        'model': model,
        'accuracy': accuracy,
        'predictions': predictions
    }
    
    return results


y_train = np.array(train_labels)
y_test = np.array(test_labels)

svm_tfidf = svm(X_train_tfidf, X_test_tfidf, y_train, y_test, "TF-IDF Features")

Training SVM with TF-IDF Features...
SVM Accuracy: 0.8889

Classification Report:
              precision    recall  f1-score   support

    Negative       0.87      0.88      0.88      3044
    Positive       0.90      0.89      0.90      3691

    accuracy                           0.89      6735
   macro avg       0.89      0.89      0.89      6735
weighted avg       0.89      0.89      0.89      6735

